In [1]:
import os
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [2]:
data_dir = 'C:/Users/이다혜/Desktop/2020-2/AI/week4/data'

### 1. Vocabulary 선정
#### TF-IDF를 수행하여 긍정, 부정 문서 내 가중치 상위 100개 단어들로 구성된 총 200개 단어 선정

In [3]:
class GenerateVocabulary:
    data = None # 전체 데이터
    
    
    # file_path의 데이터를 불러옴
    def __init__(self, file_path):
        self.data = open(file_path, encoding='utf8')
        
        
    # 전체 데이터를 읽어들여서 긍정, 부정 문서로 나눔   
    def load_data(self):
        p_doc, n_doc = '', ''
        for line in self.data.readlines():
            text, label = line.strip().split('\t')
            if label == '<P>':
                p_doc += text + ' '
            else:
                n_doc += text + ' '
        return p_doc, n_doc

    
    # 각 문서 내 단어의 Term Frequency를 count하여 {단어:TF} 형태의 dictionary로 반환
    def count_term_freq(self, p_doc, n_doc):
        p_term_freq = {}
        for i in range(len(p_doc.split(" "))):
            word = p_doc.split(" ")[i]
            if word not in p_term_freq.keys():
                p_term_freq[word] = 1
            else:
                p_term_freq[word] += 1
                
        n_term_freq = {}
        for i in range(len(n_doc.split(" "))):
            word = n_doc.split(" ")[i]
            if word not in n_term_freq.keys():
                n_term_freq[word] = 1
            else:
                n_term_freq[word] += 1
        return p_term_freq, n_term_freq

    
    # 각 문서 내 단어의 Document Frequency를 count하여 {단어:DF} 형태의 dictionary로 반환
    def count_doc_freq(self, p_doc, n_doc, p_term_freq, n_term_freq):
        p_doc_freq = {}
        for i in range(len(list(p_term_freq.keys()))):
            word = list(p_term_freq.keys())[i]
            count = 1
            if word in list(n_term_freq.keys()):
                count = 2
            p_doc_freq[word] = count
        
        n_doc_freq = {}
        for i in range(len(list(n_term_freq.keys()))):
            word = list(n_term_freq.keys())[i]
            count = 1
            if word in list(p_term_freq.keys()):
                count = 2
            n_doc_freq[word] = count
        return p_doc_freq, n_doc_freq
            
        
    # 각 문서 내 단어의 TF-IDF를 구하여 {단어:가중치} 형태의 dictionary로 반환
    def get_weights(self, p_term_freq, n_term_freq, p_doc_freq, n_doc_freq):
        N = 2 # 전체 문서 수
        
        p_term_weights = {}
        for i in range(len(p_term_freq)):
            word = list(p_term_freq.keys())[i]
            tf = list(p_term_freq.values())[i]
            df = list(p_doc_freq.values())[i]
            term_weight = tf * math.log(N/df)
            p_term_weights[word] = term_weight
            
        n_term_weights = {}
        for i in range(len(n_term_freq)):
            word = list(n_term_freq.keys())[i]
            tf = list(n_term_freq.values())[i]
            df = list(n_doc_freq.values())[i]
            term_weight = tf * math.log(N/df)
            n_term_weights[word] = term_weight
        return p_term_weights, n_term_weights
            
        
    # 각 문서 내 가중치가 높은 단어 100개씩 선정하여 총 200개 단어로 구성된 vocabulary 만듦       
    def get_vocab(self, p_term_weights, n_term_weights):
        p_vocab = sorted(p_term_weights.items(), key=lambda x : x[1], reverse=True)[:100]
        n_vocab = sorted(n_term_weights.items(), key=lambda x : x[1], reverse=True)[:100]
        vocabulary = p_vocab + n_vocab
        return vocabulary
    
    
    # 자동화 하기 위한 함수
    def doit(self):
        p_doc, n_doc = self.load_data()
        p_term_freq, n_term_freq = self.count_term_freq(p_doc, n_doc)
        p_doc_freq, n_doc_freq = self.count_doc_freq(p_doc, n_doc, p_term_freq, n_term_freq)
        p_term_weights, n_term_weights = self.get_weights(p_term_freq, n_term_freq, p_doc_freq, n_doc_freq)
        vocabulary = self.get_vocab(p_term_weights, n_term_weights)
        return vocabulary

In [4]:
voca = GenerateVocabulary(file_path='C:/Users/이다혜/Desktop/2020-2/AI/week4/data/sentiment_train.txt').doit()

In [5]:
len(voca)

200

In [6]:
voca

[('적극', 8.317766166719343),
 ('철', 6.931471805599453),
 ('사용법', 6.931471805599453),
 ('고민', 6.238324625039508),
 ('강추', 5.545177444479562),
 ('쏙', 3.4657359027997265),
 ('강추합니다', 3.4657359027997265),
 ('세심', 3.4657359027997265),
 ('여러', 3.4657359027997265),
 ('모습', 2.772588722239781),
 ('강추입니다', 2.772588722239781),
 ('배려', 2.772588722239781),
 ('좋구여', 2.772588722239781),
 ('가격싸고', 2.772588722239781),
 ('훌륭', 2.0794415416798357),
 ('완벽', 2.0794415416798357),
 ('잔', 2.0794415416798357),
 ('베리', 2.0794415416798357),
 ('당근', 2.0794415416798357),
 ('어른', 2.0794415416798357),
 ('전용', 2.0794415416798357),
 ('행복', 2.0794415416798357),
 ('전기', 2.0794415416798357),
 ('강추예요', 2.0794415416798357),
 ('제일', 2.0794415416798357),
 ('부자', 2.0794415416798357),
 ('신속', 2.0794415416798357),
 ('가르치', 2.0794415416798357),
 ('기숙사', 2.0794415416798357),
 ('쿠폰', 1.3862943611198906),
 ('안심', 1.3862943611198906),
 ('역', 1.3862943611198906),
 ('구조', 1.3862943611198906),
 ('사고', 1.3862943611198906),
 ('시험', 1.3862

### 2. NaiveBayes 분류기 학습 및 테스트를 위한 데이터셋 

In [7]:
# 분류기 학습 및 테스트를 위한 데이터셋 만들기
def get_dataset(data_dir, vocabulary):
    train_data = open(os.path.join(data_dir, 'sentiment_train.txt'), encoding='utf8')
    test_data = open(os.path.join(data_dir, 'sentiment_test.txt'), encoding='utf8')
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    
    for line in train_data.readlines():
        text, label = line.strip().split('\t')
        for word in text.split(" "):
            # 유효한 문장인지 확인 (vocabulary의 단어가 포함되지 않으면 제거)
            if word in list(vocabulary.keys()):
                X_train.append(text)
                # 편의를 위해 긍정인 경우 label 1, 부정인 경우 label 0로 인코딩 
                if label == '<P>':
                    y_train.append(1)
                else:
                    y_train.append(0)
                break 
        
    for line in test_data.readlines():
        text, label = line.strip().split('\t')
        for word in text.split(" "):
            # 유효한 문장인지 확인 (vocabulary의 단어가 포함되지 않으면 제거)
            if word in list(vocabulary.keys()):
                X_test.append(text)
                # 편의를 위해 긍정인 경우 label 1, 부정인 경우 label 0로 인코딩
                if label == '<P>':
                    y_test.append(1)
                else:
                    y_test.append(0)
                break 

    return X_train, y_train, X_test, y_test

In [8]:
X_train, y_train, X_test, y_test = get_dataset('C:/Users/이다혜/Desktop/2020-2/AI/week4/data', dict(voca))

In [9]:
X_train

['조금 작 은 감 이 있 지만 잘 쓰 ㄹ께 요',
 '통 돌 이 너무너무 조왕',
 '더 얼마나 기다리 어야 하 나요',
 '다만 치 고는 세탁물 들어가 는 양 이 좀 적 은 듯',
 '가격 배송 다 맘 에 들 는데 설치 를 우리 가 다시 하 아야 되 겠 네요',
 '그런데 어제 세탁 하 다 보 니 고정 하 았 다던 세탁기 가 혼자 서 센치를 걷 어 나오 았 어요',
 '구 정전 에 구매 하 ㄴ 것 이 라 배송이 늦 은 것 은 알 고 있 었 지만 배송 관련 하 여 안내 하 시 는 분 이 전화번호 를 알리 어 주 며 그쪽 으로 전화 하 면 정확 하 ㄴ 배송 날짜 를 알 수 있 다 하 여 전화 를 드리 었 더니만 그쪽 판매자 분 의 무 성의 하 ㄴ 전화 통화 에 화가 나 았 습니다',
 '구정 때 는 배송이 그럴려니하야지 빨 랴 야 주일 걸릴거닌까 기다리 시 어요 하 며 질문 에 똑같 은 말 만 하 는 판매자 분 의 말 등',
 '벌 ㄴ가 담당자 분 께서 이런 점 을 지적 하 여 저 아니 ㄴ 다른 분 들 까지 기분 이 상하 는 일 이 없 으면 하 여 몇 자 적 어 보내 ㅂ니다',
 '제 가 시간 이 안 되 어서 참 맞추 기 힘들엇는데도끝까지최선을 다 하 아 주 시 어서 감사 하 ㅂ니다',
 '그런데 키로 사이즈 아니 ㄴ 것 같이 넘작네여',
 '배송 일자 를 요구 사항 에 기재 하 았 는데 요청 한 날자 에 도 소식 이 없 어 연락 하 았 더니 재고 가 없 다는 황당 하 ㄴ 말 을 들 었 습니다',
 '몇 번 을 전화 하 았 지만 일주일 주문 은 월 이 ㄹ 도착 요청 한 날 월일 뒤 에 배송 되 ㄴ다 하 여 주문 취소 하 ㄴ다 하 았 더니 그 다음날 배송 되 어 오 았 더군 요',
 '지 마켓 만원 할인 쿠폰 을 이용 하 아서 싸 게 사 아서 더욱 좋 구요',
 '근데 무지 조용 하 구 예쁘데요',
 '적극 추천 하 ㅂ니다',
 '식구 많 지 않 구 특히혼자 살 시 는 분 이 아이 몸집 이 작 구 예쁘 다 하 니 딱 이 ㄴ 거 같 네요',
 '무게 도 가볍 아 

In [10]:
y_train

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,


In [11]:
X_test

['그런데 배송 일자 가 느리 어 요',
 '적극 추천 하 아요',
 '다만 물류 에서 문제 가 있 는지 일 만 에 물건 받 았 어요',
 '아 잘못 구입 하 았 냐는 생각 어떻하 냐 요',
 '아쉽 ㄴ 점 은 뚜껑 부분 이 약하 ㄴ 것 같 아 살살 다루 어야 되 겠 어요',
 '새 것 이 라서 받 았 을 때 기분 은 좋 지만 실용 성 은 영',
 '대우 제품 이미지 기업 이미지 때문 에 사실 별로 썩 이 라 생각 되 었 었 는데 상품 받 아서 사용 하 아 보 니 드럼 세탁기 에 대하 ㄴ 방송 내용 등등 사실 반반 이 었 는데 이불 까 지 빨 아서 테스트 다 하 아 보 고 나 니 기분 상쾌 하 네 요',
 '제 가 오히려 미안 하 ㄴ 거 있 죠',
 '사전 에 무슨 연락 이 라도 있 었 으면 이렇 게 화 이 나 지 ㄴ 않 을 터 이 ㄴ데 말 이 ㅂ니다',
 '배송이 이래 도 되 ㅂ니까',
 '전 에 는 대우 에 대하 ㄴ 안 좋 은 선입견 때문 에 상품 구입 을 망셜였지만 이젠 대우전자 제품 그 누구 에게 도 강력 추천 하 ㄹ 만큼 성능 도 가격 도 모두 굿 이 ㅂ니다',
 '정말 짜증 나 더라구요',
 '강추합니다',
 '통 돌 아 가 는 힘 이 약하 다',
 '사고 후회없을신껍니다',
 '밖 에 없 어서 사 았 는데 뭐 자 면서 세탁 하 ㄹ 것 은 아니 니깐 크 ㄴ 문제 는 아니 지만 그래도 좀 아쉽 어요',
 '이 이 라고 는 하 는데 기존 에 쓰 던 세탁기 보다 훨씬 작 아서 조금 실망 하 았 어요',
 '대우꺼라선지 튼튼 하 게 보이 고 성능 도 좋고배송도 빠르 ㄴ 편 이 고 배송 기사 분 들 도 친절 하 시 고 좋 네 여',
 '대우 이 라 잔 고장 도 없 을 것 같 고 전 에 있 을 때 자취 하 면서 쓰 ㄴ 거 랑 같 은 거 이 라 고르 ㄴ 것 이 ㄴ데 잘 고르 았 다고 생각 되 ㅂ니다',
 '이미 설치 하 ㄴ 후 에 뚜껑 을 열 어 보 아서 반품 하 ㄹ 수 도 없 고 그냥 사용 하 ㅂ니다',
 '꾸껑을 열 면 안 에 불 이 켜 어 지 어서 좋 았 

In [12]:
y_test

[0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

### 3. NaiveBayes 분류기 학습

In [13]:
# dictonary의 단어만으로 구성된 리스트를 만들어 countvectorizer에 넣음
voca = list(dict(voca).keys())

In [14]:
# 테스트셋의 문장을 vectorize
feature_train_X = CountVectorizer(vocabulary=voca).transform(X_train)

In [15]:
print(feature_train_X.get_shape)

<bound method spmatrix.get_shape of <744x200 sparse matrix of type '<class 'numpy.int64'>'
	with 853 stored elements in Compressed Sparse Row format>>


In [16]:
print(feature_train_X)

  (1, 89)	1
  (2, 179)	1
  (3, 106)	1
  (4, 105)	1
  (5, 103)	1
  (6, 143)	1
  (6, 180)	1
  (7, 158)	1
  (7, 159)	1
  (8, 123)	1
  (8, 145)	1
  (8, 181)	1
  (9, 90)	1
  (10, 103)	1
  (11, 102)	1
  (11, 160)	1
  (12, 102)	1
  (12, 111)	1
  (13, 29)	1
  (13, 91)	1
  (14, 92)	1
  (15, 0)	1
  (16, 93)	1
  (16, 94)	1
  (17, 95)	1
  :	:
  (724, 115)	1
  (724, 138)	1
  (724, 180)	1
  (725, 167)	1
  (726, 101)	1
  (727, 18)	1
  (728, 54)	1
  (729, 0)	1
  (730, 108)	1
  (730, 147)	2
  (731, 2)	1
  (732, 118)	1
  (733, 66)	1
  (734, 195)	1
  (735, 107)	1
  (735, 117)	1
  (735, 128)	1
  (736, 133)	1
  (737, 110)	1
  (738, 183)	1
  (739, 111)	1
  (740, 21)	1
  (741, 160)	1
  (742, 143)	1
  (743, 158)	1


In [17]:
# 분류기 학습
classifier = MultinomialNB()
classifier.fit(feature_train_X, y_train)

MultinomialNB()

### 4. NaiveBayes 분류기 성능 확인

In [18]:
feature_test_X = CountVectorizer(vocabulary=voca).transform(X_test)
print(feature_test_X.get_shape)

<bound method spmatrix.get_shape of <104x200 sparse matrix of type '<class 'numpy.int64'>'
	with 104 stored elements in Compressed Sparse Row format>>


In [19]:
print(feature_test_X)

  (0, 103)	1
  (1, 0)	1
  (2, 106)	1
  (4, 118)	1
  (4, 134)	1
  (6, 86)	1
  (6, 133)	2
  (7, 82)	1
  (8, 129)	1
  (9, 150)	1
  (10, 185)	1
  (11, 100)	1
  (12, 6)	1
  (13, 134)	1
  (14, 33)	1
  (15, 118)	1
  (16, 108)	1
  (17, 61)	1
  (19, 151)	1
  (21, 61)	1
  (24, 132)	1
  (25, 185)	1
  (26, 17)	1
  (27, 108)	1
  (30, 102)	1
  :	:
  (75, 126)	1
  (76, 103)	1
  (77, 179)	1
  (78, 63)	1
  (79, 130)	1
  (80, 118)	1
  (80, 178)	1
  (81, 102)	1
  (83, 124)	1
  (84, 103)	1
  (85, 29)	1
  (86, 130)	1
  (88, 4)	1
  (89, 106)	1
  (90, 138)	1
  (92, 190)	1
  (93, 105)	1
  (94, 105)	1
  (95, 9)	1
  (96, 106)	1
  (96, 127)	1
  (98, 183)	1
  (99, 113)	1
  (102, 71)	1
  (103, 108)	1


In [20]:
predictions = classifier.predict(feature_test_X).tolist()

In [21]:
from sklearn.metrics import accuracy_score

print('Accuracy: %.2f' % accuracy_score(y_test, predictions))

Accuracy: 0.85
